# RAG - Structuration des Requêtes

Les systèmes RAG se sont révélés très puissants pour enrichir les modèles de génération en langage naturel, en leur permettant d'accéder à des bases de documents textuels. Cependant, la véritable force des RAG réside dans leur capacité à interagir non seulement avec des documents statiques, mais également avec des bases de données complexes et dynamiques. Cette connexion offre un potentiel immense, car elle permet d'interroger des données structurées via des requêtes, ouvrant la porte à des réponses plus précises et à jour. Structurer ces requêtes de manière efficace devient alors un enjeu clé dans la mise en œuvre des systèmes RAG capables d'aller au-delà du texte brut, en accédant à des informations spécifiques et pertinentes à partir de vastes ensembles de données.

![](https://i.ibb.co/ysfSWZY/RAG-10.png)

**Objectifs :** Dans ce notebook, nous explorerons les techniques et les approches pour structurer ces requêtes, garantissant une interaction fluide et pertinente avec les bases de données dans un contexte de génération augmentée par la récupération.


#### 0. Préparation

Installons d'abord quelques dépendances nécessaires :

In [ ]:
!pip install pytube youtube-transcript-api

Dans ce notebook, nous utiliserons l'API d'OpenAI, vous devez vous inscrire sur [**OpenAI**](https://beta.openai.com/signup/) et [**récupérer une clé d'API**](https://platform.openai.com/api-keys).   

Pour surveiller les appels faits aux modèles LLMs, nous vous conseillons aussi d'utiliser LangSmith. Vous devrez [**générer une clé d'API LangSmith**](https://smith.langchain.com/settings) puis activer le traçage.

Une fois les clés obtenues, ajoutez-les à vos variables d'environnement :

In [6]:
import os

# Remplacez "<votre_cle_openai>" par votre clé API OpenAI
os.environ["OPENAI_API_KEY"] = "<votre_cle_openai>"

# Remplacez "<votre_cle_langsmith>" par votre clé API LangSmith
os.environ['LANGCHAIN_TRACING_V2'] = "false"
os.environ['LANGCHAIN_API_KEY'] = "<votre_cle_langsmith>"

### 1. Exemple : Base de vidéos Youtube

Prenons l'exemple d'une base de données de vidéos. Grâce à LangChain, nous pouvons récupérer le transcript d'une vidéo Youtube ainsi que les métadonnées associées :

In [ ]:
from langchain_community.document_loaders import YoutubeLoader

yt_url = "https://www.youtube.com/watch?v=pbAd8O1Lvm4"

docs = YoutubeLoader.from_youtube_url(yt_url, add_video_info=True).load()

docs[0].metadata

A partir de ces données, nous pourrions construire une base de données de vidéos avec les attributs suivants :

| Titre | Description | Nombre de Vues | Date de Publication | Durée | Transcription |
|-------|-------------|----------------|---------------------|-------|---------------|
| ...   | ...         | ...            | ...                 | ...   | ...           |


Afin de générer des requêtes structurées, nous devons d'abord définir notre schéma de requête. Nous pouvons voir que chaque document possède un titre, un nombre de vues, une date de publication, et une durée en secondes. Supposons que nous avons construit un index qui nous permet d'effectuer une recherche non structurée sur le contenu et le titre de chaque document, et d'utiliser le filtrage par plages sur le nombre de vues, la date de publication, et la durée.

Pour commencer, nous allons créer un schéma avec des attributs explicites pour les valeurs minimum et maximum du nombre de vues, de la date de publication, et de la durée de la vidéo afin de pouvoir filtrer selon ces critères. Nous ajouterons également des attributs distincts pour les recherches effectuées sur le contenu de la transcription par rapport au titre de la vidéo.

In [8]:
import datetime
from typing import Optional

from langchain_core.pydantic_v1 import BaseModel, Field


class YoutubeDBSearch(BaseModel):
    """Search over a database of videos."""

    content_search: str = Field(
        ...,
        description="Similarity search query applied to video transcripts.",
    )
    title_search: str = Field(
        ...,
        description=(
            "Alternate version of the content search query to apply to video titles. "
            "Should be succinct and only include key words that could be in a video "
            "title."
        ),
    )
    min_view_count: Optional[int] = Field(
        None,
        description="Minimum view count filter, inclusive. Only use if explicitly specified.",
    )
    max_view_count: Optional[int] = Field(
        None,
        description="Maximum view count filter, exclusive. Only use if explicitly specified.",
    )
    earliest_publish_date: Optional[datetime.date] = Field(
        None,
        description="Earliest publish date filter, inclusive. Only use if explicitly specified.",
    )
    latest_publish_date: Optional[datetime.date] = Field(
        None,
        description="Latest publish date filter, exclusive. Only use if explicitly specified.",
    )
    min_length_sec: Optional[int] = Field(
        None,
        description="Minimum video length in seconds, inclusive. Only use if explicitly specified.",
    )
    max_length_sec: Optional[int] = Field(
        None,
        description="Maximum video length in seconds, exclusive. Only use if explicitly specified.",
    )

    def pretty_print(self) -> None:
        for field in self.__fields__:
            if getattr(self, field) is not None and getattr(self, field) != getattr(
                self.__fields__[field], "default", None
            ):
                print(f"{field}: {getattr(self, field)}")

Nous pouvons ensuite utiliser ce schéma pour transformer une requête utilisateur en une requête structurée qui exploite les différents attributs de notre base de données :
```

In [9]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI

def get_youtube_db_query(question):
    system = """You are an expert at converting user questions into database queries. \
    You have access to a database of tutorial videos about a software library for building LLM-powered applications. \
    Given a question, return a database query optimized to retrieve the most relevant results.

    If there are acronyms or words you are not familiar with, do not try to rephrase them."""

    prompt = ChatPromptTemplate.from_messages(
        [("system", system), ("human", "{question}")]
    )
    llm = ChatOpenAI(model="gpt-3.5-turbo-0125", temperature=0)
    structured_llm = llm.with_structured_output(YoutubeDBSearch)
    query_analyzer = prompt | structured_llm
    return query_analyzer.invoke(question)

In [ ]:
# ---------- Test ----------

query = get_youtube_db_query("A full course for langchain, 1 hour minimum")

query.pretty_print()

### 2. Exercice : Base de données - Livres

A vous de jouer !

Vous devez transformer une entrée utilisateur non structurée en une requête structurée pour une bibliothèque en ligne. Voici la base de données de livres pour laquelle vous devez structurer les requêtes :

| Title | Author | Category | Publication Date | Number of Pages | Summary |
|-------|--------|----------|------------------|-----------------|---------|
| ...   | ...    | ...      | ...              | ...             | ...     |

**Objectif :** Créer une fonction qui transforme une entrée utilisateur en une requête structurée pour cette base de données de livres.


**Instructions :**
- Commencez par définir un schéma de requête `LibraryDBSearch` pour cette base de données :
    - La recherche doit obligatoirement contenir des valeurs pour les attributs `Summary` et `Category`.
    - Toutes les autres valeurs de recherche sont optionnelles
- Créez la fonction `get_library_db_query` :
    - Prend en entrée une requête utilisateur non structurée
    - Retourne une recherche (un objet `LibraryDBSearch`)
    - Utilisez les [Structured Output](https://python.langchain.com/v0.1/docs/modules/model_io/chat/structured_output/) pour utiliser votre schéma `LibraryDBSearch`.


In [16]:
from typing import Optional

from langchain_core.pydantic_v1 import BaseModel, Field



class LibraryDBSearch(BaseModel):

    summary_search: str = Field(
        ...,
        description="(Required) Similarity search query applied to book summaries.",
    )
    
    category_search: str = Field(
        ...,
        description="(Required) Search query applied to the category of the book.",
    )

    title_search: Optional[str] = Field(
        None,
        description="Search query applied to the title of the book.",
    )

    author_search: Optional[str] = Field(
        None,
        description="Search query applied to the author of the book.",
    )

    earliest_publish_date: Optional[str] = Field(
        None,
        description="Earliest publish date filter, inclusive. Only use if explicitly specified.",
    )
    latest_publish_date: Optional[str] = Field(
        None,
        description="Latest publish date filter, exclusive. Only use if explicitly specified.",
    )

    min_page_count: Optional[int] = Field(
        None,
        description="Minimum page count filter, inclusive. Only use if explicitly specified.",
    )
    max_page_count: Optional[int] = Field(
        None,
        description="Maximum page count filter, exclusive. Only use if explicitly specified.",
    )

    def pretty_print(self) -> None:
        for field in self.__fields__:
            if getattr(self, field) is not None and getattr(self, field) != getattr(
                self.__fields__[field], "default", None
            ):
                print(f"{field}: {getattr(self, field)}")

def get_library_db_query(question):

    system = """You are an expert at converting user questions into database queries. \
    You have access to a database of books. \
    Given a question, return a database query optimized to retrieve the most relevant results. \
    Must include a search query for the book summary and the category of the book.
    """

    prompt = ChatPromptTemplate.from_messages(
        [("system", system), ("human", "{question}")]
    )
    llm = ChatOpenAI(model="gpt-3.5-turbo-0125", temperature=0)
    structured_llm = llm.with_structured_output(LibraryDBSearch)
    query_analyzer = prompt | structured_llm
    return query_analyzer.invoke(question)

In [ ]:
# ---------- Test ----------

def get_query_and_print(question):
    print('Question:', question)
    query = get_library_db_query(question)
    query.pretty_print()
    print('\n')

get_query_and_print("Je cherche un livre de science-fiction écrit par Arthur Clarke publié après 1965.")
get_query_and_print("Je cherche un livre avec une histoire de magiciens.")
get_query_and_print("Je cherche un livre historique.")